In [2]:
import numpy as np

# 9.1 Inductive Bias

### 9.1.1 Inverse Problems

Most machine learning problems are *inverse problems*:\
Given a conditional distribution, we may easily generate samples of observations from the distribution. In ML, we almost always care to solve the *inverse* of this problem - i.e. given a sample of observations, determine the distribution that generated them. The fundamental difficulty with this is that infinitely many distributions could have generated any sample of observations. 

The preference for one distribution over others is called *inductive bias*. This umbrella term is lent to many flavors of useful bias such as domain knowledge or medium knowledge (like knowing that the medium of images possesses translation invariance), among many others.

### 9.1.2 No Free Lunch Theorem

This theorem states that every learning algorithm is as good as any other when averaged over all possible problems...

In finding good inductive biases, we want biases that are appropriate to the broad class of problems that our models will be applied to in practice. For a given application, better results are obtained by incorporating stronger inductive biases for the specific applications of interest.

### 9.1.3 Symmetry and Invariance

In many applications, good predictions are *invarian* under one or more transformations of the input variables. For instance, computer vision models should be able to identify objects regardless of where they appear in an image (*translation invariance*). 

Transformations that leave crucial properties unchanged represent *symmetries* in an ML context. For instance, a cat should be identifiable as a cat regardless of whether it is changed by a transformation of scale or position in an image. Thus, the dominant features that identify it as a cat are *symmetric* under these transformations. The set of all transformations that correspond to a particular symmetry comprise a *group*...

A *group* consists of a set of elements $A, B, C, ...$ together with a *binary operation* for compsing pairs of elements, denoted $A \circ B$.\
Groups have the following axioms:
1. **Closed** - Groups are closed under their binary operation: $$A\circ B \in \mathcal{G}, \ \forall A, B \in \mathcal{G}$$
2. **Associative**: $$(A\circ B) \circ C = A\circ (B \circ C), \ \forall A, B, C \in \mathcal{G}$$
3. **Identity**: $$\exists I \in \mathcal{G} : A \circ I = I \circ A = A, \ \forall A \in \mathcal{G}$$
4. **Inverse**: $$\exists A^{-1} \in \mathcal{G} : A\circ A^{-1} = A^{-1} \circ A = I, \ \forall A \in \mathcal{G}$$

### 9.1.4 Equivariance

This is a generalization of invariance in which the output of the network is itself transformed when the inputs are transformed.\
E.g. a network that classifies pixels in an image as either being in the foreground or the background should translate the segmentations in the same way that the input is translated. 

Letting $\bf I$ be an input, $S$ be the network operation, and $T$ be the transformation operation, then equivariance holds if:
$$S(T(\mathbf I)) = T(S(\mathbf I))$$

More generally, equivariance may still apply if a different operation $\tilde{T}$ is applied to the output such that $S(T(\mathbf I)) = \tilde{T}(S(\mathbf I))$\
(Does $\tilde{T}$ need to be isomorphic to $T$ or something??)

Invariance is a special case of equivariance in which $\tilde{T}$ is the identity operator, i.e. : $$S(T(\mathbf{I})) = S(\mathbf{I})$$

# 9.2 Weight Decay

For a weight vector $w$ and error function $E(w)$, the $l2$ regularized error is given by: $$\tilde{E}(w) = E(w) + \frac \lambda 2 w^\intercal w$$
Here the coefficient $1/2$ is included for convenience in differentiation.\
This type of regularization is called *weight decay* because it encourages weight values to decay towards zero. This principal should apply to $l1$ regularization as well, though perhaps more pronouncedly... The derivative is then:
$$\nabla \tilde E (w) = \nabla E(w) + \lambda w$$


## 9.2.1 Consistent Regularizers

Weight decay can violate *consistency* - i.e. it can make networks variant to benign transformations of inputs and outputs (such as affine transformations).

For example, if we apply an affine transformation to the input of a simple MLP, $x \mapsto ax + b$, then we can ensure the output is unchanged by making complimentary linear transformations to the weights and biases in the input layer:
$$ w_{ji} \mapsto \tilde w_{ji} = \frac{1}{a} w_{ji}, \ \ \ \ w_{j0} \mapsto \tilde w_{j0} = w_{j0} - \frac{b}{a}\sum w_{ji}$$

If we use the generice weight decay regularizer, then the penalty for the transformed network will be inequal to the penatly for the original network - introducing a regularization bias.\
Consider the diagonal elements of $\frac{\lambda}{2}\tilde w^\intercal \tilde w$:
$$
\text{Weights: } \ \ \frac{\lambda}{2}\tilde w_{i}^\intercal \tilde w_{i} = \frac{\lambda}{2a^2} w_{i}^\intercal w_i \\ \ \\
\text{Biases: } \ \ \frac{\lambda}{2}\tilde w_{0}^\intercal w_0 = \frac{\lambda}{2} \bigg[ \sum_j \big(w_{j0} - \frac{b}{a}\sum_i w_{ji}\big)^2 \bigg]
$$

Clearly, the weights and biases have ***inequal influence*** on the regularization penalty. Moreover, they impose a penalty that is *not linearly transformed* in the same fashion as the input.

A ***Consistent Regularizer*** is invariant to re-scaling of weights and shifts of biases. Such a regularizer is:
$$\frac{\lambda_1}{2} \sum_{w\in W_1}w^2 + \frac{\lambda_2}{2}\sum_{w\in W_2}w^2$$
Where $W_1$ denotes the *first laryer* and $W_2$ denotes the *second layer* - biases are **excluded** from the summations. Finally, we scale $\lambda_1$ and $\lambda_2$ by $a^{1/2}$ to ensure consistency under weights scaling.

The book kind of hand-waves this away, as well as the specification of consistent biases... 

## 9.2.2 Generalized Weight Decay

A generalization of the simple quadratic regularizer (i.e. $l2$ penalty) is:
$$\Omega (w) = \frac{\lambda}{2} \sum_{j=1}^M |w_j|^q$$
At $q=2$ this penalty becaome the $l2$ penalty and at $q=1$ it becomes the $l1$ penalty.

# 9.3 Learning Curves

## 9.3.1 Early Stopping

We can stop iterative training algorithms like SGD early to prevent overfitting. This is effectively another form of regularization. Indeed, it may in theory select similar weights to those that weight decay would select by stopping training when the weights reach $\hat w$ (the regularized optimum) instead of $w_{ML}$ (the maximum-likelihood global optimum).

## 9.3.2 Double Descent

Deep neural networks run-contrary to conventional statistical-learning wisdom that suggests over-parameterization as a cause for overfitting (i.e. high-variance). This is a bit perplexing, since neural nets will often have parameter counts well past the point of saturation for their datasets. 

***Double Descent*** is a term coined to describe how deep learning models can have different learning-curve behaviors for unseen test/validation data than for training data. Imagine a simple GLM (i.e. a singe-layer NN). When the parameter count for the GLM is low, we may expect poor performance on test data (the model is too inflexible, it has high bias, poor representation). As the parameter count increases, the test/validation loss should improve (better representation/generalization, bis-variance trade-off is good). As the parameter count reaches saturation, we would expect the model to over-fit to the training data and begin to perform poorly on test/validation data once more (too flexible, high variance, poor generalization to unseen data). Deep neural networks have been shown not to behave this way. Belkin *et al.* 2019 trained a deep network (ResNet18) at varying levels of model complexity (i.e. parameter counts). Initially, as the parameter count increased, training loss and validation loss evolved in a fashion typical to that of the GLM example. This they termed the "*Classical Regime: Bias-Variance Trade-Off*". The validation loss even increased as expected. However, as the parameter count continued to increase, even well past the point of the training loss converging to zero, the validation loss began to decrease again - and it *continued to decrease without end*.

Thus, past some point of complexity (i.e. parameter count), the training entered the "*Modern Regime*" in which the larger model is always better. ***Double Descent*** is commonly taken to refer to this regime, i.e. we achieve double descent once we make the validation loss begin to descrease again (seemingly independent of the training loss).

The work cited by Bishop on this suggests that double descent occurs roughly where the number of parameters in the model is large enough to fit the training data **exactly**. The ***Effective Model Complexity*** is the model size which can achieve zero training error.

Similar behavior is observable simply in the course of model training. There is some intuition for this if we think of the training iterations as effectively adding parameters to the model - and therefore, making it more complex over the course of training.

This is all a bit mysterious, and I feel it must be attributable to the *inductive biases* learnt by deep neural networks...

An irony of double descent is that in some cases increasing the training dataset size can actually ***decrease performance*** on unseen data!

# 9.4 Parameter Sharing

This is another form of regularization that *groups together* parameters and requires all parameters within a group to have the same or similar values. This *decreases the degrees of freedom* within the model to be fewer than the number of *connections* within the network. (Do convolutions kind of do this?)

## 9.4.1 Soft Weight Sharing

This introduces a regularization term that encourages groups of weights to have similar values. Consider $l2$ regularization. The penalty term encourages all weight values to decrease towards zero, some more than others, but all in the same direction. The soft weight sharing approach encourages the weights to form *several* groups rather than one group at zero. It does this by considering a probability distribution over a *mixture of Gaussians*. The means $\mu_j$, variances $\sigma_j^2$, and mixing coefficients $\pi_j$ are all adjustable parameters than the model *can learn*.

This mixture parameterizes a probability distribution for the weights, $p(w)$. The negative log-likelihood of this distribution is then used as the regularization penalty, $\Omega(w)$.\
 **NOTE:** Chapter 2 explains how the $l2$ penalty can be understood as a log-likelihood for the prior distribution of parameter weights $p(w)$ via Baye's theorem (where each weight $w_i$ is a IID unit Gaussian RV).
$$
p(w) = \prod_i \bigg[\sum_{j=1}^K \pi_j \mathcal{N}(w_i | \mu_j, \sigma_j^2) \bigg] \\ \ \\
\Omega(w) = - \sum_i \ln \bigg(\sum_{j=1}^K \pi_j \mathcal{N}(w_i|\mu_j, \sigma_j^2)\bigg)
$$

Then the error function is given by:
$$ \tilde E(w) = E(w) + \lambda\Omega(w)$$

So, the fundamental difference between this approach and the standard $l2$ penalty, is that the prior distribution $p(w)$ is no longer assumed to be a joint PDF of IID unit Gaussians, but a *Gaussian mixture*.\
SGD can learn the parameters of the mixture by evaluating the derivatives of $\Omega(w)$ w.r.t. *all* learnable parameters. This is facilitated by considering each $\pi_j$ as a prior probability for each $j^{th}$ Gaussian to have generated a weight value. Then, the following posterior probabilities may be introduced:
$$ 
\gamma_j(w_i) = \frac{\pi_j \mathcal{N}(w_i|\mu_j, \sigma_j^2)}{\sum_k \pi_k \mathcal{N}(w_i|\mu_k, \sigma_k^2)}
= \frac{P(\text{Component }j) P(w_i| \text{Component }j)}{P(w_i)}
$$

The numerator here is the probability of observing $w_i$ generated by Gaussian $j$. The denominator is the marginal likelihood of observing $w_i$ (i.e. from *any* Gaussian in the mixture). And $\gamma_j(w_i)$ is the posterior probability of $w_i$ belonging to Gaussian $j$. 

The partial derivative of the loss w.r.t. weight $w_i$ is then:
$$\partial w_i: \ \ \ \frac{\partial E(w)}{\partial w_i} + \lambda \sum_j \gamma_j (w_i) \frac{(w_i - \mu_j)}{\sigma_j^2}$$
For the means:
$$\partial \mu_j: \ \ \ \lambda \sum_i \gamma_j(w_i)\frac{(\mu_j - w_i)}{\sigma_j^2}$$
The variances are a bit trickier because they are contrained to be positive via $\sigma^2 = \exp(\epsilon_j)$, then the partial derivative is computed for epsilon as:
$$\partial \epsilon_j: \ \ \ \frac{\lambda}{2} \sum_i \gamma_j(w_i) \bigg( 1 - \frac{(w_i - \mu_j)^2}{\sigma_j^2}\bigg)$$

Then we can express the mixing coefficient $\pi_j$ as a softmax function of continuous variables $\eta_j$. This enables smooth differentiation beacuse $\pi_j$ is bound by $0$ and $1$:
$$\partial \eta_j: \ \ \ \lambda\sum_i\big(\pi_j - \gamma_j(w_i)\big)$$

# Weight Decay in Optimization

When reading this section, I realized that I've never seen these weight decay penalties added to a loss function in practice. It seems that this is because modern implementations *don't* add weight decay penalties to the loss function directly, but indirectly through optimization algorithms for training (e.g. ADAM and ADAMW).

"Decoupled Weight Decay Regularization" (2019) introduced ADAMW `torch.optim.AdamW` which is ADAM with *decoupled* weight decay. I'll attempt to explain what this is.

Apparently weight decay as first proposed (in 1988!) allows the weights to decay exponentially as:
$$w_{t+1} = (1 - \lambda)w_t - \eta \nabla f_t(w_t)$$
Where $\lambda$ defines the rate of the weight decay per step and $\nabla f_t(w_t)$ is the batch gradient.\
For SGD, this *is equivalent to $l2$ regularization*!\
Specifically, they are equivalent when the $l2$ penalty is set to $\lambda/\eta$\
This is part of why $l2$ regularization has come to be referred to as "weight decay"

The ADAMW team pointed out that weight decay *is not equivalent to* $l2$ regularization when ADAM is used for learning. 

In ADAMW weight decay is explicitly decoupled from the loss function and is implemented as another term in the training algorithm - they literally just add one term to the ADAM update:
$$
w_i^{(\tau)} = w^{(\tau - 1)} - \text{ADAM Update } - \eta \lambda w^{(\tau - 1)} \\
 \ = w_i^{(\tau - 1)} - \eta \frac{\hat{s_i}^{(\tau)}}{\sqrt{\hat{r_i}^{(\tau)}} + \epsilon}
- \eta \lambda w^{(\tau - 1)}
$$

So, it seems like this is why I don't see discussions about weight decay / regularization w.r.t. loss functions. It's just baked into optimization (pretty simply).

# 9.5 Residual Connections

Residual connections simply add the inputs to a layer to the outputs of that layer before feeding forward to the next layer of the network. (Noting ofc that we may add residual connections around blocks or entire networks, like Bengio 2003). Let $F(\cdot)$ denote the function expressed by a network layer (i.e. linear transformation(s) and non-linear activation(s)). Then given inputs $z_1$, the layer's outputs with a residual connection are simply:
$$z_2 = \text F(z_1) + z_1$$

The term "*residual*" hints at the fact that the residual block expressed by $\text F (\cdot)$ *learns the residual* between the output and the input:
$$\text F (z_1) = z_2 - z_1$$

If we chain the residuals all the way to the final network output and allow $\text F(\cdot)$ to express the *entire network* then: $$\text F(x) = y - x$$
We can see that the network is learning the *residual* between the inputs and the outputs.

This *smooths out* error surfaces and helps to reduce vanishing/exploding gradients.

Residual connections can effectively ***decompose*** deep networks into *linear combinations* of more shallow networks. Thus, when the deeper networks lose or obscure updates in lower layers via poor gradients in higher layers, we still have the shallower updates that get updated.\
Consider a three layer network:
$$
z_1 = \text F_1(x) + x \\
z_2 = \text F_2 (z_1) + z_1 \\
\hat y = \text F_3(z_2) + z_2
$$
Then we may express $\hat y$ as:
$$
\begin{align*}
& \hat y = \text F_3(\text F_2( \text F_1(x) + x) + \text F_1(x) + x) \\
& \ \ + \text F_2(\text F_1(x) + x) \\
& \ \ + \text F_1(x) + x
\end{align*}
$$


This is a linear combination of three networks, each of increasing depth!

### Residual Connections and Gradients
I think a really crucial observation that explains this smoothing power is left out of the book. That is, if we add a residual connection, then during the backwards pass the gradient gets passed directly trhough the residual connection to everything that came *before* the residual block. Thus, the influence of the layers wihin the residual block on the updates of layers before the residual block is moderated by the residual connection. If gradients in the residual block vanish, then we still get information with which to update weights in earlier layers of the network via the residual connection.

For example, consider a two layer network with two activation functions. Without a residual connection, this may look like:
$$
z^{(1)} = h^{(1)}\big(w^{(1)}x\big) = h^{(1)}(a^{(1)})\\ \ \\
\hat y = h^{(2)}\big(w^{(2)}z^{(1)}) = h^{(2)}(a^{(2)})
$$
Where $a^{(1)}$ and $a^{(2)}$ denote the preactivations to each layer.
Then for some loss $E(\hat y)$, the partial derivative of the loss at observation $i$ w.r.t. $w^{(1)}_i$ via the chain rule is:
$$
\frac{\partial E(\hat y_i)}{\partial w^{(1)}_i} = \frac{\partial E(\hat y_i)}{\partial \hat y_i} \frac{\partial \hat y_i}{\partial a^{(2)}} \frac{\partial a^{(2)}}{\partial z^{(1)}} \frac{\partial z^{(1)}}{\partial a^{(1)}} \frac{\partial a^{(1)}}{\partial w^{(1)}_i}
$$

And per SGD we would update $w^{(1)}_i$ as:
$$
w^{(1)}_i \leftarrow w^{(1)}_i - \eta \bigg(\frac{\partial E(\hat y_i)}{\partial \hat y_i} \frac{\partial \hat y_i}{\partial a^{(2)}} \frac{\partial a^{(2)}}{\partial z^{(1)}} \frac{\partial z^{(1)}}{\partial a^{(1)}} \frac{\partial a^{(1)}}{\partial w^{(1)}_i}\bigg)
$$
Suppose the partial derivative $\partial \hat y_i / \partial a^{(2)}$ was near zero (e.g. a vanishing gradient when $h^{(2)}$ is the Softmax). Then the update term would collapse to zero.

Now, if we add a residual connection, the gradient has two "paths":
$$
\hat y = h^{(2)}(a^{(2)}) + z^{(1)} \\ \ \\
\frac{\partial E(\hat y_i)}{\partial w^{(1)}_i} = \frac{\partial E(\hat y_i)}{\partial \hat y_i} \frac{\partial \hat y_i}{\partial a^{(2)}} \frac{\partial a^{(2)}}{\partial z^{(1)}} \frac{\partial z^{(1)}}{\partial a^{(1)}} \frac{\partial a^{(1)}}{\partial w^{(1)}_i} + 
\frac{\partial E(\hat y_i)}{\partial \hat y_i} \frac{\partial \hat y_i}{\partial z^{(1)}} \frac{\partial z^{(1)}}{\partial a^{(1)}} \frac{\partial a^{(1)}}{\partial w^{(1)}_i}
$$

Now even if the gradient vanishes in the second layer, we can still update $w_i^{(1)}$ as:
$$
w^{(1)}_i \leftarrow w^{(1)}_i - \eta \bigg(\frac{\partial E(\hat y_i)}{\partial \hat y_i} \frac{\partial \hat y_i}{\partial z^{(1)}} \frac{\partial z^{(1)}}{\partial a^{(1)}} \frac{\partial a^{(1)}}{\partial w^{(1)}_i} \bigg)
$$

This *smooths* the error surface and the training process by reducing the influence of volatile gradients in higher layers of the network on the weight updates in lower layers of the network.

This is equivalent to the network decomposition illustration.

# 9.6 Model Averaging

We may use *ensembles* of seveal models working to produce the same prediction by averaging their predictions together. Such a predicted distribution is:
$$p(y|x) = \frac{1}{L}\sum_{l=1}^L p_l(y|x)$$
Where $p_l(y|x)$ is the output of model $l$, and $L$ is the total number of models.

Bagged models are a special case of ensemble models (well known in forests) that train several models on bootstrapped samples from the same training dataset and then averages over their predictions. 

Likewise, we may train several models of varied architectures on the same training data and average over them as to yield an ensemble prediction.

Then of course we have boosting (ala GBMs) which train models successively upon the errors (residuals or gradients) of each prior model.

## 9.6.1 Dropout

Dropout (randomly zeroing out weights during forward passes) can be viewed as implicitly performing model averaging over *exponentially* many models *without* training multiple models individually.

Each network in a forward pass with dropout has a random subset of trainable parameters of the original full network. Thus, we may think of it as a random subnetwork, or a *new model*. Thus, predictions drawn from the fully trained model are in a sense *ensemble* predictions of the form:
$$p(y|x) = \sum_R p(R) p(y|x, R)$$
Where $R$ is the matrix that applies the dropout mask, and summing over $R$ is summing over the space of all achievable masks. $p(R)$ is the probability of observing a mask, and $p(y|x, R)$ is the predictive distribution of the network with mask $R$.\
This probability grows exponentially in complexity as the size of the mask $R$ increases. However, it may be approximated with a small number of sample masks to achieve good posterior predictive distribution estimates. This procedure is known as ***Monte Carlo Dropout***

Dropout is also sort of Bayesian sampling. For a fully connected network with $M$ hidden parameters, we could create $2^M$ different subnetworks by selectively zeroing-out (or "*ablating*") parameters. A fully Bayesian treatment would make predictions by averaging over all possible $2^M$ networks, with the output of each being weighted by its posterior probability. Dropout approximates such a fully-Bayesian mmodel by giving an *equal posterior probability* to each possible model. 

Dropout also reduces overfitting by reducing the risk of individual parameters (and pathways) from becoming overly tuned to any particular aspect of the problem, which is particularly a risk when considering noise or high-leverage data. By randomly ablating parameters, we force all parameters to generalize more.